In [1]:
from mindspore import context
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [2]:
from mindsponge import Sponge
from mindsponge import Molecule
from mindsponge import ForceField
from mindsponge import DynamicUpdater
from mindsponge import SimulationCell
from mindsponge.control import VelocityVerlet, Langevin, BerendsenBarostat
from mindsponge.function import VelocityGenerator
from mindsponge.callback import WriteH5MD, RunInfo
from mindsponge.optimizer import SteepestDescent

In [3]:
system = Molecule(template='water.spce.yaml')
system.set_pbc_box([0.4, 0.4, 0.4])

Molecule<>

如果定义了周期性的盒子，那么还可以使用repeat_box功能来扩展/复制系统，比如可以这样将系统复制1000份

In [4]:
system.repeat_box([10, 10, 10])

Molecule<>

得到的分子系统的示意图如下所示：
![](../../docs/tutorial_b05.png)

In [5]:
potential = ForceField(system, parameters='SPCE')

In [6]:
opt = SteepestDescent(system.trainable_params(), 1e-6)

In [7]:
sim = SimulationCell(system, potential, cutoff=1.0)
md = Sponge(sim, optimizer=opt)

[WARNING] KERNEL(19979,7f683a92f740,python):2022-08-15-14:32:25.115.623 [mindspore/ccsrc/plugin/device/gpu/kernel/cuda_impl/cuda_class/cuda_class_common.h:55] CalShapesSizeInBytes] For 'Neg', the shapes[0] is ( )
[WARNING] KERNEL(19979,7f683a92f740,python):2022-08-15-14:32:25.117.226 [mindspore/ccsrc/plugin/device/gpu/kernel/cuda_impl/cuda_class/cuda_class_common.h:55] CalShapesSizeInBytes] For 'Floor', the shapes[0] is ( )
[WARNING] KERNEL(19979,7f683a92f740,python):2022-08-15-14:32:26.056.566 [mindspore/ccsrc/plugin/device/gpu/kernel/cuda_impl/cuda_class/cuda_class_common.h:55] CalShapesSizeInBytes] For 'Square', the shapes[0] is ( )
[WARNING] KERNEL(19979,7f683a92f740,python):2022-08-15-14:32:26.058.358 [mindspore/ccsrc/plugin/device/gpu/kernel/cuda_impl/cuda_class/cuda_class_common.h:55] CalShapesSizeInBytes] For 'Erfc', the shapes[0] is ( )
[WARNING] KERNEL(19979,7f683a92f740,python):2022-08-15-14:32:26.059.095 [mindspore/ccsrc/plugin/device/gpu/kernel/cuda_impl/cuda_class/cuda_cl

In [8]:
run_info = RunInfo(10)
md.run(100, callbacks=[run_info])

[WARNING] KERNEL(19979,7f683a92f740,python):2022-08-15-14:32:30.001.118 [mindspore/ccsrc/plugin/device/gpu/kernel/cuda_impl/cuda_class/cuda_class_common.h:55] CalShapesSizeInBytes] For 'Neg', the shapes[0] is ( )


Step: 0, E_pot: -706514.3, 
Step: 10, E_pot: -780552.5, 
Step: 20, E_pot: -782711.0, 
Step: 30, E_pot: -782719.3, 
Step: 40, E_pot: -782720.56, 
Step: 50, E_pot: -782722.25, 
Step: 60, E_pot: -782720.25, 
Step: 70, E_pot: -782721.56, 
Step: 80, E_pot: -782719.44, 
Step: 90, E_pot: -782720.6, 


In [9]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(system.coordinate.shape, system.atom_mass)

[WARNING] KERNEL(19979,7f683a92f740,python):2022-08-15-14:32:31.673.448 [mindspore/ccsrc/plugin/device/gpu/kernel/gpu_kernel_factory.cc:93] ReducePrecision] Kernel [StandardNormal] does not support int64, cast input 0 to int32.
[WARNING] PRE_ACT(19979,7f683a92f740,python):2022-08-15-14:32:31.674.619 [mindspore/ccsrc/plugin/device/gpu/optimizer/reduce_precision_fusion.cc:84] Run] Reduce precision for [StandardNormal] input 0


In [10]:
opt = DynamicUpdater(system,
                     integrator=VelocityVerlet(system),
                     thermostat=Langevin(system, temp),
                     barostat=BerendsenBarostat(system, 1),
                     velocity=velocity,
                     time_step=5e-4)
md = Sponge(sim, optimizer=opt)

In [11]:
md.change_optimizer(opt)

In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_b05.h5md', save_freq=10)

In [13]:
md.run(1000, callbacks=[run_info, cb_h5md])

[WARNING] KERNEL(19979,7f683a92f740,python):2022-08-15-14:32:35.690.275 [mindspore/ccsrc/plugin/device/gpu/kernel/gpu_kernel_factory.cc:93] ReducePrecision] Kernel [StandardNormal] does not support int64, cast input 0 to int32.
[WARNING] PRE_ACT(19979,7f683a92f740,python):2022-08-15-14:32:35.712.062 [mindspore/ccsrc/plugin/device/gpu/optimizer/reduce_precision_fusion.cc:84] Run] Reduce precision for [StandardNormal] input 0


Step: 0, E_pot: -782718.3, E_kin: 11166.619, E_tot: -771551.7, Temperature: 298.55188, Pressure: 3412.2046, Volume: 64.0
Step: 10, E_pot: -779667.44, E_kin: 7620.862, E_tot: -772046.56, Temperature: 203.75214, Pressure: 1441.239, Volume: 64.0301
Step: 20, E_pot: -779614.06, E_kin: 9060.289, E_tot: -770553.75, Temperature: 242.23682, Pressure: 2623.5615, Volume: 64.06473
Step: 30, E_pot: -781919.06, E_kin: 10508.436, E_tot: -771410.6, Temperature: 280.95462, Pressure: 4070.1924, Volume: 64.10864
Step: 40, E_pot: -780328.8, E_kin: 8303.462, E_tot: -772025.4, Temperature: 222.00221, Pressure: 1207.0321, Volume: 64.15063
Step: 50, E_pot: -781403.94, E_kin: 10017.07, E_tot: -771386.9, Temperature: 267.8174, Pressure: 1501.1423, Volume: 64.17863
Step: 60, E_pot: -782917.94, E_kin: 10618.373, E_tot: -772299.56, Temperature: 283.89392, Pressure: 1124.8983, Volume: 64.20259
Step: 70, E_pot: -782474.06, E_kin: 10259.326, E_tot: -772214.75, Temperature: 274.2944, Pressure: 155.06824, Volume: 64.2